In [1]:
import numpy as np
import pandas as pd
import string

In [2]:
data = pd.read_csv("Language Detection.csv")
df = pd.DataFrame(data)

In [3]:
df.head()

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English


In [4]:
#Verilerdeki gurultuleri temizleyelim. Bir dilin tespit edilmesinde buyuk ve kucuk harfler etkili degildir. Ayni zamanda noktalama isaretleri de etkili degildir. bundan dolayil;
#noktalama isaretlerini kaldiralim ve buyuk harfle yazilmis olan yazilari kucuk harfe cevirelim.

In [5]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [6]:
trash = string.punctuation + '0123456789'

def remove_noise(text):
    for not_important in trash:
        text = text.replace(not_important, '')
    
    text = text.lower()
    return text
    
df['Text'] = df['Text'].apply(remove_noise)


In [7]:
df.head()

,Text,Language
0,nature in the broadest sense is the natural p...,English
1,nature can refer to the phenomena of the physi...,English
2,the study of nature is a large if not the only...,English
3,although humans are part of nature human activ...,English
4,the word nature is borrowed from the old fren...,English


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X = df['Text']
Y = df['Language']

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y ,test_size=0.3, random_state=42)

In [11]:
from sklearn import feature_extraction

In [12]:
vec1 = feature_extraction.text.TfidfVectorizer(ngram_range=(1,2),analyzer= 'char')
vec2 = feature_extraction.text.TfidfVectorizer(ngram_range=(1,3),analyzer= 'char')
vec3 = feature_extraction.text.TfidfVectorizer(ngram_range=(1,4),analyzer= 'char')
vec4 = feature_extraction.text.TfidfVectorizer(ngram_range=(1,2),analyzer= 'word')
vec5 = feature_extraction.text.TfidfVectorizer(ngram_range=(1,3),analyzer= 'word')
vec6 = feature_extraction.text.TfidfVectorizer(ngram_range=(1,4),analyzer= 'word')

In [13]:
from sklearn import pipeline
from sklearn import linear_model

In [14]:
model_pipe1 = pipeline.Pipeline([('vec', vec1), ('clf', linear_model.LogisticRegression())])
model_pipe2 = pipeline.Pipeline([('vec', vec2), ('clf', linear_model.LogisticRegression())])
model_pipe3 = pipeline.Pipeline([('vec', vec3), ('clf', linear_model.LogisticRegression())])
model_pipe4 = pipeline.Pipeline([('vec', vec4), ('clf', linear_model.LogisticRegression())])
model_pipe5 = pipeline.Pipeline([('vec', vec5), ('clf', linear_model.LogisticRegression())])
model_pipe6 = pipeline.Pipeline([('vec', vec6), ('clf', linear_model.LogisticRegression())])

In [15]:
model_pipe1.fit(X_train, Y_train)
model_pipe2.fit(X_train, Y_train)
model_pipe3.fit(X_train, Y_train)
model_pipe4.fit(X_train, Y_train)
model_pipe5.fit(X_train, Y_train)
model_pipe6.fit(X_train, Y_train)

Pipeline(steps=[('vec', TfidfVectorizer(ngram_range=(1, 4))),
                ('clf', LogisticRegression())])

In [16]:
model_pipe1.classes_

array(['Arabic', 'Danish', 'Dutch', 'English', 'French', 'German',
       'Greek', 'Hindi', 'Italian', 'Kannada', 'Malayalam', 'Portugeese',
       'Russian', 'Spanish', 'Sweedish', 'Tamil', 'Turkish'], dtype=object)

In [17]:
pred1 = model_pipe1.predict(X_test)
pred2 = model_pipe2.predict(X_test)
pred3 = model_pipe3.predict(X_test)
pred4 = model_pipe4.predict(X_test)
pred5 = model_pipe5.predict(X_test)
pred6 = model_pipe6.predict(X_test)

In [18]:
from sklearn import metrics

In [19]:
print('char(1,2) ====> ' + str(metrics.accuracy_score(Y_test,pred1)))
print('char(1,3) ====> ' + str(metrics.accuracy_score(Y_test,pred2)))
print('char(1,4) ====> ' + str(metrics.accuracy_score(Y_test,pred3)))
print('word(1,2) ====> ' + str(metrics.accuracy_score(Y_test,pred4)))
print('word(1,3) ====> ' + str(metrics.accuracy_score(Y_test,pred5)))
print('word(1,4) ====> ' + str(metrics.accuracy_score(Y_test,pred6)))

char(1,2) ====> 0.9745325596389426
char(1,3) ====> 0.9825918762088974
char(1,4) ====> 0.9832366215344939
word(1,2) ====> 0.9564796905222437
word(1,3) ====> 0.9526112185686654
word(1,4) ====> 0.9477756286266924


In [20]:
# Bu egitim sonuclarina gore karakter bazli egitimler daha iyi sonuclar verdi ve ngram_range (yani her seferinde yan yana olusuna bakilan karakter gruplari) arttirildiginda
# karakter bazli egitimlerde modelin performansi iyilesti. Tam aksine kelime bazli egitimlerde ise modelin performansi kotulesti. Cunku, dillerin algilanmasi icin kelimelerin 
# hangi dilden oldugunu tespit etmek yeterlidir. Bundan dolayi karakterlerin olusturdugu butune bakmak yeterli olur. Kelimelerin olusturdugu butunler daha cok cumleleri ve 
# paragraflari olusturduklari icin kelime butunleri; cumle yorumlamalari, chat tabanli modeller gibi yerlerde kullanilmak icin daha uygundurlar.